# Data Cleaning & Pre-processing

#### 1. Addressing NaN, Nulls: 
- capture meaning information found from NaN into Nones, NAs, and 0s

#### 2. Merging datasets
- Housing Price dataset: qualitative and quantitative information on 2624 housing transactions from 2006 - 2010 in 28 neighborhoods in Ames, Iowa *(82 columns, 78 features)*

- Ames Real Estate dataset: more granular location info (addresses, map ref #) and some qualitative on each transaction (central air, building materials used, etc) *(91 columns)*

- stored in GCP blob

- *Merged on ‘PID’ — 172 columns, many duplicate features.*


- Geolocation data from Google API, stored in GCP blob

#### 3. Dropping some columns 

**4. Next**: [Feature Engineering notebook](http://localhost:8890/notebooks/3_feature_engineering.ipynb)

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [2]:
#import the raw data here
data = pd.read_csv('./data/raw/Ames_HousePrice.csv').iloc[:, 1:]

In [3]:
print(data.shape)

(2580, 81)


In [4]:
#data.info()

In [5]:
#data.describe()

# Data Cleaning Part 1 —
## *handling null values in ames dataset*


In [6]:
#investigate nulls
data.isnull().sum().sort_values(ascending = False).head(15)

PoolQC          2571
MiscFeature     2483
Alley           2412
Fence           2055
FireplaceQu     1241
LotFrontage      462
GarageFinish     129
GarageQual       129
GarageYrBlt      129
GarageCond       129
GarageType       127
BsmtExposure      71
BsmtFinType2      70
BsmtFinType1      69
BsmtCond          69
dtype: int64

#### LotFrontage

In [7]:
#Lot Frontage
data['LotFrontage'].value_counts().head()

#Examining only the null values
lot_nulls = data[data['LotFrontage'].isna()] #a lot of properties with no linear feat of street connected to property. 

#These seem to be real properties, let's set the NaNs to the average of the corresponding type of :otConfig. 
lot_nulls.head()

,PID,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,909176150,856,126000,30,RL,NaN,7890,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,SWISU,Norm,Norm,1Fam,1Story,6,6,1939,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,238.0,Unf,0.0,618.0,856.0,GasA,TA,Y,SBrkr,856,0,0,1.0,0.0,1,0,2,1,TA,4,Typ,1,Gd,Detchd,1939.0,Unf,2.0,399.0,TA,TA,Y,0,0,0,0,166,0,NaN,NaN,NaN,0,3,2010,WD,Normal
13,535105100,1394,159000,20,RL,NaN,9500,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1963,1963,Gable,CompShg,Plywood,Plywood,BrkFace,247.0,TA,TA,CBlock,Gd,TA,No,BLQ,609.0,Unf,0.0,785.0,1394.0,GasA,Gd,Y,SBrkr,1394,0,0,1.0,0.0,1,1,3,1,TA,6,Typ,2,Gd,Attchd,1963.0,RFn,2.0,514.0,TA,TA,Y,0,76,0,0,185,0,NaN,NaN,NaN,0,7,2009,WD,Normal
18,534152050,1610,205000,20,RL,NaN,10603,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6,7,1977,2001,Gable,CompShg,Plywood,Plywood,BrkFace,28.0,TA,TA,PConc,TA,TA,Mn,ALQ,1200.0,Unf,0.0,410.0,1610.0,GasA,Gd,Y,SBrkr,1610,0,0,1.0,0.0,2,0,3,1,Gd,6,Typ,2,TA,Attchd,1977.0,RFn,2.0,480.0,TA,TA,Y,168,68,0,0,0,0,NaN,NaN,NaN,0,2,2010,WD,Normal
27,533221090,1573,177500,160,FV,NaN,2117,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,Twnhs,2Story,6,5,2000,2000,Gable,CompShg,MetalSd,MetalSd,BrkFace,216.0,Gd,TA,PConc,Gd,TA,No,GLQ,378.0,Unf,0.0,378.0,756.0,GasA,Ex,Y,SBrkr,769,804,0,0.0,0.0,2,1,3,1,Gd,5,Typ,0,NaN,Detchd,2000.0,Unf,2.0,440.0,TA,TA,Y,0,32,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
28,534128010,2090,200000,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NWAmes,PosN,Norm,1Fam,2Story,7,6,1973,1973,Gable,CompShg,HdBoard,HdBoard,Stone,240.0,TA,TA,CBlock,Gd,TA,Mn,ALQ,859.0,BLQ,32.0,216.0,1107.0,GasA,Ex,Y,SBrkr,1107,983,0,1.0,0.0,2,1,3,1,TA,7,Typ,2,TA,Attchd,1973.0,RFn,2.0,484.0,TA,TA,Y,235,204,228,0,0,0,NaN,NaN,Shed,350,11,2009,WD,Normal


In [8]:
# Group lots by configuration
data.groupby(['LotConfig']).agg({'LotFrontage' : 'mean'})

,LotFrontage
LotConfig,
Corner,81.468023
CulDSac,56.456790
FR2,59.413793
FR3,79.300000
Inside,66.633846


In [9]:
#impute np.NaN with average for LotConfig of that type. 

data.loc[(data['LotFrontage'].isna()) & (data['LotConfig'] == 'Corner'), 'LotFrontage'] = 81.47
data.loc[(data['LotFrontage'].isna()) & (data['LotConfig'] == 'CulDSac'), 'LotFrontage'] = 56.46
data.loc[(data['LotFrontage'].isna()) & (data['LotConfig'] == 'FR2'), 'LotFrontage'] = 59.41
data.loc[(data['LotFrontage'].isna()) & (data['LotConfig'] == 'FR3'), 'LotFrontage'] = 79.30
data.loc[(data['LotFrontage'].isna()) & (data['LotConfig'] == 'Inside'), 'LotFrontage'] = 66.63

data['LotFrontage'].isna().sum()

0

In [10]:
#taking a look at outliers
data['LotFrontage'].sort_values().tail()

74      168.0
1146    174.0
527     195.0
2008    200.0
981     313.0
Name: LotFrontage, dtype: float64

In [11]:
#take note of maybe drop
LF_maybe_drop = data[data["LotFrontage"] == 313]

#### Alley

In [12]:
data['Alley'].isnull().sum() #2412 null values
data['Alley'].value_counts()

Grvl    105
Pave     63
Name: Alley, dtype: int64

In [13]:
#make note to one hot encode this later a having alley or no alley
data['Alley'] = data['Alley'].replace(np.nan, 'None')

In [14]:
data['Alley'].value_counts()

None    2412
Grvl     105
Pave      63
Name: Alley, dtype: int64

#### MasVnrType &  MasVnrArea

In [15]:
data[data['MasVnrType'].isna()].head()

,PID,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
175,528439010,1460,247000,20,RL,87.00,10037,Pave,None,Reg,Lvl,AllPub,Corner,Gtl,Somerst,Feedr,Norm,1Fam,1Story,8,5,2006,2007,Hip,CompShg,VinylSd,VinylSd,NaN,NaN,Gd,TA,PConc,Ex,TA,No,GLQ,666.0,Unf,0.0,794.0,1460.0,GasA,Ex,Y,SBrkr,1460,0,0,0.0,0.0,2,0,3,1,Gd,6,Typ,1,Gd,Attchd,2006.0,Fin,2.0,480.0,TA,TA,Y,0,20,0,0,0,0,NaN,NaN,NaN,0,8,2008,WD,Normal
358,528462040,1428,182000,20,FV,95.00,11639,Pave,None,Reg,Lvl,AllPub,Corner,Gtl,Somerst,Norm,Norm,1Fam,1Story,7,5,2007,2008,Gable,CompShg,CemntBd,CmentBd,NaN,NaN,Gd,TA,PConc,Gd,TA,No,Unf,0.0,Unf,0.0,1428.0,1428.0,GasA,Ex,Y,SBrkr,1428,0,0,0.0,0.0,2,0,3,1,Gd,6,Typ,0,NaN,Attchd,2007.0,Fin,2.0,480.0,TA,TA,Y,0,120,0,0,0,0,NaN,NaN,NaN,0,12,2008,New,Partial
753,533208040,1241,199900,120,FV,35.00,4274,Pave,Pave,IR1,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,TwnhsE,1Story,7,5,2006,2007,Gable,CompShg,VinylSd,VinylSd,NaN,NaN,Gd,TA,PConc,Gd,TA,No,GLQ,1106.0,Unf,0.0,135.0,1241.0,GasA,Ex,Y,SBrkr,1241,0,0,1.0,0.0,1,1,1,1,Gd,4,Typ,0,NaN,Attchd,2007.0,Fin,2.0,569.0,TA,TA,Y,0,116,0,0,0,0,NaN,NaN,NaN,0,11,2007,New,Partial
787,528240070,1960,216500,60,RL,66.63,7851,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,2002,2002,Gable,CompShg,VinylSd,VinylSd,NaN,NaN,Gd,TA,PConc,Gd,TA,No,GLQ,625.0,Unf,0.0,235.0,860.0,GasA,Ex,Y,SBrkr,860,1100,0,1.0,0.0,2,1,4,1,Gd,8,Typ,2,TA,BuiltIn,2002.0,Fin,2.0,440.0,TA,TA,Y,288,48,0,0,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal
871,907260030,1725,198500,60,RL,70.00,8749,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2002,2002,Gable,CompShg,VinylSd,VinylSd,NaN,NaN,Gd,TA,PConc,Gd,TA,No,Unf,0.0,Unf,0.0,840.0,840.0,GasA,Ex,Y,SBrkr,840,885,0,0.0,0.0,2,1,3,1,Gd,6,Typ,0,NaN,Attchd,2002.0,RFn,2.0,550.0,TA,TA,Y,0,48,0,0,0,0,NaN,NaN,NaN,0,11,2009,WD,Normal


In [16]:
#none is meaningful here and so is 0
data['MasVnrType'] = data['MasVnrType'].replace(np.nan, 'None')
data['MasVnrArea'] = data['MasVnrArea'].replace(np.nan, 0)

#### Basement related features

In [17]:
data['BsmtQual'].value_counts() #missing NA for no basement

TA    1166
Gd    1077
Ex     188
Fa      78
Po       2
Name: BsmtQual, dtype: int64

In [18]:
#these all receive NA if no basement
basement_to_na = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']

#these receive 0 sqft if basement is NA
basement_to_zero = ['BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFinSF1', 'BsmtFullBath','BsmtHalfBath']

In [19]:
#Set NaN to NAs
for i in basement_to_na:
     data[i] = data[i].replace(np.nan, 'None')

In [20]:
#data[data['BsmtFinSF1'].isna()] #the missing records are for NA basement properties. Setting to sqft value to zero.

In [21]:
#set basement sqft to 0 IF basement is NA

for i in basement_to_zero:
    data[i] = data[i].replace(np.nan, 0)

#### Fireplace Quality

In [22]:
print(data['FireplaceQu'].unique())
print(data['Fireplaces'].unique())

['Gd' nan 'Ex' 'TA' 'Fa' 'Po']
[1 0 2 3 4]


In [23]:
#number of houses w/o fireplace = fireplace_qu np.nan 
len(data[(data['FireplaceQu'].isna()) & (data['Fireplaces'] < 1)]) 

1241

In [24]:
#perform replacement 
data['FireplaceQu'] = data['FireplaceQu'].replace(np.nan, 'None') #NA for no fireplace

#### Garage related features

In [25]:
#create two lists, one for catagorical and one for numeric

garage_to_na = ['GarageType','GarageFinish','GarageQual','GarageCond']
garage_to_0 = ['GarageYrBlt','GarageCars', 'GarageArea']

In [26]:
print(data[['GarageType','GarageFinish','GarageQual','GarageCond']].isnull().sum())
print()
print(data[['GarageYrBlt','GarageCars', 'GarageArea']].isnull().sum())

GarageType      127
GarageFinish    129
GarageQual      129
GarageCond      129
dtype: int64

GarageYrBlt    129
GarageCars       1
GarageArea       1
dtype: int64


In [27]:
#replace categorical columns with NA 
for i in garage_to_na: 
    data[i] = data[i].replace(np.nan, 'None')

In [28]:
#replace ordinal columns with 0
for i in garage_to_0: 
    data[i] = data[i].replace(np.nan, 0)

#### Pools

In [29]:
data[['PoolQC']].isnull().sum()

PoolQC    2571
dtype: int64

In [30]:
#making pool a binary category and dropping pool area
data[['Pool']] = data[['PoolQC']].replace(np.nan,0)
data = data.drop(['PoolQC', 'PoolArea'], axis =1)

#### Fence

In [31]:
print(data['Fence'].isnull().sum())

2055


In [32]:
data['Fence'] = data['Fence'].replace(np.nan, 'None') #turn NaN into NA
print(data['Fence'].isnull().sum())

0


#### misc_feature

In [33]:
data['MiscFeature'].isnull().sum()

2483

In [34]:
data = data.drop('MiscFeature', axis = 1)

#### Electrical

In [35]:
data[data['Electrical'].isnull()]

,PID,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,Fence,MiscVal,MoSold,YrSold,SaleType,SaleCondition,Pool
2442,916386080,1394,167500,80,RL,73.0,9735,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,Timber,Norm,Norm,1Fam,SLvl,5,5,2006,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0.0,Unf,0.0,384.0,384.0,GasA,Gd,Y,NaN,754,640,0,0.0,0.0,2,1,3,1,Gd,7,Typ,0,None,BuiltIn,2007.0,Fin,2.0,400.0,TA,TA,Y,100,0,0,0,0,None,0,5,2008,WD,Normal,0


In [36]:
data['Electrical'].unique()

array(['SBrkr', 'FuseF', 'FuseA', 'FuseP', nan], dtype=object)

In [37]:
data['Electrical'] = data['Electrical'].replace(np.nan, 'SBrkr') #took most common category under electrical

#### Final Null Check

In [38]:
data_cleaned = data

In [39]:
data_cleaned.isnull().sum()

PID              0
GrLivArea        0
SalePrice        0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
Street           0
Alley            0
LotShape         0
LandContour      0
Utilities        0
LotConfig        0
LandSlope        0
Neighborhood     0
Condition1       0
Condition2       0
BldgType         0
HouseStyle       0
OverallQual      0
OverallCond      0
YearBuilt        0
YearRemodAdd     0
RoofStyle        0
RoofMatl         0
Exterior1st      0
Exterior2nd      0
MasVnrType       0
MasVnrArea       0
ExterQual        0
ExterCond        0
Foundation       0
BsmtQual         0
BsmtCond         0
BsmtExposure     0
BsmtFinType1     0
BsmtFinSF1       0
BsmtFinType2     0
BsmtFinSF2       0
BsmtUnfSF        0
TotalBsmtSF      0
Heating          0
HeatingQC        0
CentralAir       0
Electrical       0
1stFlrSF         0
2ndFlrSF         0
LowQualFinSF     0
BsmtFullBath     0
BsmtHalfBath     0
FullBath         0
HalfBath         0
BedroomAbvGr

# Data Cleaning Part 2 —
## *merging housing price data, real estate data, and geolocation*
At this point, the only necessary data from the real_estate_data is the address. We will merge that data with the housing price data, along with long lat data for each property.

### First, let's clean the real estate dataset and drop some columns as well

In [40]:
#import real estate dataset
re = pd.read_csv('./data/raw/Ames Real Estate Data.csv').iloc[:, 1:]\

re.isnull().sum().sort_values()

GeoRefNo            0
DwlgNo_S            0
BldgNo_S            0
AcreNt_S            0
AcreGr              0
AcreX_S1            0
Source              0
ImpAcX2S            0
ImpAcX1S            0
LndAcX1S            0
ValType             0
TtlVal_AsrYr        0
OthAc_S             0
LndAc_S             0
ParclRel            0
ImpAc_S             0
SchD_S              0
Tier                0
ClassPr_S           0
ClassSc_S           0
Date                0
MA_Ownr1            0
Range               0
TxD_S               0
Legal_Pr            0
MA_City            14
MA_State           14
MA_Zip1            14
MA_Line1           14
PA-Strt            20
Prop_Addr          20
PA-Nmbr            20
PA-StSfx           98
ZngCdPr           106
Inst1_No         1857
Inst1_Mo         1907
Inst1_Yr         1907
Rcrd_Mo          1907
Rcrd_Yr          1907
Inst1TPr         2677
RoofMatl         4308
Ext1             4308
Ext2             4308
MasVnrType       4308
Heating          4308
Fireplaces

In [41]:
#list of columns to drop
re_to_drop = ['Tier','Range','ZngCdPr','ZngCdSc','ZngOLPr','ZngOLSc','ClassPr_S','ClassSc_S','Legal_Pr','SchD_S',
'TxD_S','MA_Ownr1','MA_Ownr2','MA_Line1','MA_Line2','MA_City','MA_State','MA_Zip1','MA_Zip2','Rcrd_Yr','Rcrd_Mo','Inst1_No','Inst1_Yr',
'Inst1_Mo','Inst1TPr','LndAc_S','X1TPr_D','X1TSc_D','X2TPr_D','X2TSc_D','X1TPr_S','X1TSc_S',
'X2TPr_S','X2TSc_S','X2TSc_S', 'YrSold_YYYY', 'MoSold_MM', 'SalePrice', 'SaleType', 'SaleCond', 
    'PA-PreD', 'PA-PostD', 'PA-UnTyp', 'PA-UntNo', 'NmbrBRs', 'MilVal_S', 'HSTtl_S', 'MilVal_D', 'HSTtl_D', 'TotRmsAbvGrd']

#drop emmm due to high amount of nulls (19k out of 20k)
re = re.drop(columns=re_to_drop)

In [42]:
re.isnull().sum().sort_values()

GeoRefNo           0
ParclRel           0
Date               0
DwlgNo_S           0
BldgNo_S           0
AcreNt_S           0
AcreGr             0
AcreX_S1           0
Source             0
ImpAcX1S           0
LndAcX1S           0
ValType            0
ImpAcX2S           0
ImpAc_S            0
OthAc_S            0
TtlVal_AsrYr       0
Prop_Addr         20
PA-Strt           20
PA-Nmbr           20
PA-StSfx          98
Fireplaces      4308
GarageArea      4308
Cars            4308
GarYrBlt        4308
GarageType      4308
PoolArea        4308
TtlBsmtSF       4308
MasVnrType      4308
Central Air     4308
Heating         4308
Ext2            4308
Ext1            4308
RoofMatl        4308
HouseStyle      4308
YrBuilt         4308
BldgType        4308
ParType         4308
LotArea         4308
Neighborhood    4308
GLA             4308
Foundation      4308
dtype: int64

In [43]:
more_re_to_drop = ['Fireplaces', 'PoolArea', 'GarageArea', 'GarYrBlt', 'Cars', 
                   'GarageType', 'TtlBsmtSF','Heating', 'Central Air', 'MasVnrType', 'Ext2', 
                   'Ext1', 'RoofMatl','HouseStyle','YrBuilt','BldgType','ParType','LotArea',
                   'Neighborhood','GLA','Foundation']

re = re.drop(columns=more_re_to_drop)
re.isnull().sum().sort_values()

GeoRefNo         0
ParclRel         0
DwlgNo_S         0
BldgNo_S         0
AcreNt_S         0
AcreGr           0
Date             0
ImpAcX2S         0
AcreX_S1         0
LndAcX1S         0
ValType          0
TtlVal_AsrYr     0
OthAc_S          0
ImpAc_S          0
ImpAcX1S         0
Source           0
Prop_Addr       20
PA-Nmbr         20
PA-Strt         20
PA-StSfx        98
dtype: int64

In [44]:
re.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22213 entries, 0 to 22212
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   GeoRefNo      22213 non-null  int64  
 1   Prop_Addr     22193 non-null  object 
 2   ImpAc_S       22213 non-null  int64  
 3   OthAc_S       22213 non-null  int64  
 4   TtlVal_AsrYr  22213 non-null  int64  
 5   ValType       22213 non-null  object 
 6   LndAcX1S      22213 non-null  int64  
 7   ImpAcX1S      22213 non-null  int64  
 8   ImpAcX2S      22213 non-null  int64  
 9   AcreX_S1      22213 non-null  float64
 10  AcreGr        22213 non-null  float64
 11  AcreNt_S      22213 non-null  float64
 12  BldgNo_S      22213 non-null  int64  
 13  DwlgNo_S      22213 non-null  int64  
 14  ParclRel      22213 non-null  object 
 15  PA-Nmbr       22193 non-null  object 
 16  PA-Strt       22193 non-null  object 
 17  PA-StSfx      22115 non-null  object 
 18  Date          22213 non-nu

### Let the merge begin!

In [45]:
#this dataframe holds the merging column, and the data we need associated with that column
necessary_info = re[['GeoRefNo','Prop_Addr']]

In [46]:
#we are merging the two datasets by PID (in housing price data) and GeoRefNo (necessary_info)

merged_house_data = data_cleaned.merge(necessary_info, how = 'left',
                                              left_on= 'PID', right_on='GeoRefNo')

In [47]:
#upload geodata
geo_data = pd.read_csv('./data/geo/address_lat_long.csv', index_col = False)

housing_price_data = merged_house_data.merge(geo_data, how = 'left',
                         left_on ='Prop_Addr', right_on = 'address')

housing_price_data = housing_price_data.drop(['Unnamed: 0','address','GeoRefNo'],axis = 1)
housing_price_data.head()

,PID,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,Fence,MiscVal,MoSold,YrSold,SaleType,SaleCondition,Pool,Prop_Addr,lat,long
0,909176150,856,126000,30,RL,81.47,7890,Pave,None,Reg,Lvl,AllPub,Corner,Gtl,SWISU,Norm,Norm,1Fam,1Story,6,6,1939,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,238.0,Unf,0.0,618.0,856.0,GasA,TA,Y,SBrkr,856,0,0,1.0,0.0,1,0,2,1,TA,4,Typ,1,Gd,Detchd,1939.0,Unf,2.0,399.0,TA,TA,Y,0,0,0,0,166,None,0,3,2010,WD,Normal,0,436 HAYWARD AVE,42.017780,-93.651452
1,905476230,1049,139500,120,RL,42.00,4235,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,5,5,1984,1984,Gable,CompShg,HdBoard,HdBoard,BrkFace,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,Y,SBrkr,1049,0,0,1.0,0.0,2,0,2,1,Gd,5,Typ,0,None,Attchd,1984.0,Fin,1.0,266.0,TA,TA,Y,0,105,0,0,0,None,0,2,2009,WD,Normal,0,3416 WEST ST,42.024697,-93.664186
2,905476230,1049,139500,120,RL,42.00,4235,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,5,5,1984,1984,Gable,CompShg,HdBoard,HdBoard,BrkFace,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,Y,SBrkr,1049,0,0,1.0,0.0,2,0,2,1,Gd,5,Typ,0,None,Attchd,1984.0,Fin,1.0,266.0,TA,TA,Y,0,105,0,0,0,None,0,2,2009,WD,Normal,0,3416 WEST ST,42.024697,-93.664186
3,911128020,1001,124900,30,C (all),60.00,6060,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,5,9,1930,2007,Hip,CompShg,MetalSd,MetalSd,None,0.0,Gd,TA,BrkTil,TA,TA,No,ALQ,737.0,Unf,0.0,100.0,837.0,GasA,Ex,Y,SBrkr,1001,0,0,0.0,0.0,1,0,2,1,Gd,5,Typ,0,None,Detchd,1930.0,Unf,1.0,216.0,TA,Po,N,154,0,42,86,0,None,0,11,2007,WD,Normal,0,320 S 2ND ST,42.021389,-93.614855
4,535377150,1039,114000,70,RL,80.00,8146,Pave,None,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,2Story,4,8,1900,2003,Gable,CompShg,MetalSd,MetalSd,None,0.0,Gd,Gd,BrkTil,Fa,TA,No,Unf,0.0,Unf,0.0,405.0,405.0,GasA,Gd,Y,SBrkr,717,322,0,0.0,0.0,1,0,2,1,TA,6,Typ,0,None,Detchd,1940.0,Unf,1.0,281.0,TA,TA,N,0,0,168,0,111,None,0,5,2009,WD,Normal,0,1524 DOUGLAS AVE,42.038070,-93.612065


In [48]:
# Street was only necessary to get lat and long, no longer necessary
housing_price_data.drop('Prop_Addr',axis = 1, inplace = True)

In [49]:
housing_price_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2624 entries, 0 to 2623
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PID            2624 non-null   int64  
 1   GrLivArea      2624 non-null   int64  
 2   SalePrice      2624 non-null   int64  
 3   MSSubClass     2624 non-null   int64  
 4   MSZoning       2624 non-null   object 
 5   LotFrontage    2624 non-null   float64
 6   LotArea        2624 non-null   int64  
 7   Street         2624 non-null   object 
 8   Alley          2624 non-null   object 
 9   LotShape       2624 non-null   object 
 10  LandContour    2624 non-null   object 
 11  Utilities      2624 non-null   object 
 12  LotConfig      2624 non-null   object 
 13  LandSlope      2624 non-null   object 
 14  Neighborhood   2624 non-null   object 
 15  Condition1     2624 non-null   object 
 16  Condition2     2624 non-null   object 
 17  BldgType       2624 non-null   object 
 18  HouseSty

### save file

In [50]:
housing_price_data.to_csv('./data/clean/cleaned.csv',index = False)